# Scraper institutes
This notebook is a monthly scraper used to retrieve information about condition in detention centers in Italy. To do so, it uses the id numbers of the various detention centers to navigate to the dedicated webpages with Selenium, store locally the html code of the page and then parse it using BeautifuSoup. The information is then stored in a pandas dataframe and saved as a csv file.

In [1]:
import pandas as pd
import requests
import datetime
from bs4 import BeautifulSoup
import asyncio
from playwright.async_api import async_playwright
from time import sleep
from pathlib import Path

In [2]:
# Get current date
current_month = datetime.datetime.now().strftime("%Y-%m")
current_day = datetime.datetime.now().strftime("%Y-%m-%d")

In [3]:
# Collect institutes id numbers
df_institutes = pd.read_csv(f'../outputs/clean/institutes_info_most_recent.csv')
prison_ids = df_institutes['id_istituto'].tolist()

In [4]:
# Function to grab the html code of the page
async def get_html(prison_id, current_day):

    dest = Path(f"../outputs/raw/snapshots/{current_day}_{prison_id}.html")

    if dest.exists() : #... load it from file
        print(f"Already have {dest}, loading!")
        page_html = open(dest).read()
    else:
        BASE_URL = "https://www.giustizia.it/giustizia/it/dettaglio_scheda.page?s="
        playwright = await async_playwright().start()
    
        try:
            browser = await playwright.firefox.launch()
            context = await browser.new_context(viewport={'width': 1280, 'height': 800})
            page = await context.new_page()
            await page.goto(f"{BASE_URL}{prison_id}")
            print("Fetching " + prison_id)
            page_html = await page.content()
            
            # Stores html code in dest
            # dest.write_text(page_html)
        finally:
            await browser.close()
            await playwright.stop()
    
    return page_html

In [5]:
# Function to extract institute name and type
def extract_institute_details(soup):
    institute_name = soup.find('h1', {'class': 'titoloIstituto'}).text.strip()
    institute_type = soup.find('h3', {'class': 'titoloIstituto'}).text.strip()
    return institute_name, institute_type

# Function to extract capacity table data
def extract_capacity_data(soup):
    table_capienze = soup.find_all('table')[0]
    rows = table_capienze.find_all('tr')
    if len(rows) > 1:  # Ensure there are rows in the table
        cells = rows[1].find_all('td') 
        posti_regolamentari = int(cells[0].text.strip())
        posti_non_disponibili = int(cells[1].text.strip())
        totale_detenuti = int(cells[2].text.strip())
    else:
        posti_regolamentari = posti_non_disponibili = totale_detenuti = 0
    return posti_regolamentari, posti_non_disponibili, totale_detenuti

# Function to extract updated date
def extract_updated_date(soup, header_text):
    target_span = soup.find('h2', text=header_text)
    if target_span:
        span = target_span.find_next_sibling('span')
        return span.text.strip() if span else 'NA'
    return 'NA'

def extract_personnel_details(soup, header_text):
    target_span = soup.find('h2', text=header_text)
    if target_span:
        try:
            div = target_span.find_next('div', {'class': 'listaContenutiComplessi'})
            spans = div.find_all('span', {'class': 'valoreSottocampo'})
            return spans[0].text.strip(), spans[1].text.strip(), spans[2].text.strip()
        except:
            return 'NA', 'NA', 'NA'
    return 'NA', 'NA', 'NA'

# Function to extract staff table data
def extract_staff_data(soup):
    table_staff = soup.find_all('table')[1]
    cells = table_staff.find_all('td')
    polizia_penitenziaria_effettivi = int(cells[0].text.strip())
    polizia_penitenziaria_previsti = int(cells[1].text.strip())
    amministrativi_effettivi = int(cells[2].text.strip())
    amministrativi_previsti = int(cells[3].text.strip())
    educatori_effettivi = int(cells[4].text.strip())
    educatori_previsti = int(cells[5].text.strip())
    return (polizia_penitenziaria_effettivi, polizia_penitenziaria_previsti,
            amministrativi_effettivi, amministrativi_previsti,
            educatori_effettivi, educatori_previsti)

# Function to extract last update date
def extract_date_of_last_update(soup):
    # Police staff
    target_span= soup.find('h2', text='personale polizia penitenziaria aggiornato al')
    try:
        span = target_span.find_next_sibling('span')
        personale_polizia_aggiornato_al = span.text.strip()
    except:
        personale_polizia_aggiornato_al = 'NA'

    # Administrative staff
    target_span= soup.find('h2', text='personale amministrativo aggiornato al')
    try:
        span = target_span.find_next_sibling('span')
        personale_amministrativo_aggiornato_al = span.text.strip()
    except:
        personale_amministrativo_aggiornato_al = 'NA'

    return personale_polizia_aggiornato_al, personale_amministrativo_aggiornato_al
    

In [6]:
# Function to extract institute data
def get_prison_data(soup, current_day):

    institute_name, institute_type = extract_institute_details(soup)
    posti_regolamentari, posti_non_disponibili, totale_detenuti = extract_capacity_data(soup)
    dati_aggiornati_al = extract_updated_date(soup, 'dati aggiornati al ')
    asl, first_name_asl, last_name_asl = extract_personnel_details(soup, 'Responsabile ASL per il carcere')
    first_name, last_name, role = extract_personnel_details(soup, 'Direttore')
    (polizia_penitenziaria_effettivi, polizia_penitenziaria_previsti, amministrativi_effettivi, amministrativi_previsti, educatori_effettivi, educatori_previsti) = extract_staff_data(soup)
    personale_polizia_aggiornato_al, personale_amministrativo_aggiornato_al = extract_date_of_last_update(soup)

    prison_data = {
            'id': prison_id,
            'nome': institute_name,
            'tipo': institute_type,
            'posti_regolamentari': posti_regolamentari,
            'posti_non_disponibili': posti_non_disponibili,
            'posti_occupati': totale_detenuti,
            'posti_aggiornati_al': dati_aggiornati_al,
            'asl': asl,
            'nome_responsabile_asl': first_name_asl,
            'cognome_responsabile_asl': last_name_asl,
            'nome_direttore': first_name,
            'cognome_direttore': last_name,
            'ruolo_direttore': role,
            'personale_polizia_effettivi': polizia_penitenziaria_effettivi,
            'personale_polizia_previsti': polizia_penitenziaria_previsti,
            'personale_amministrativi_effettivi': amministrativi_effettivi,
            'personale_amministrativi_previsti': amministrativi_previsti,
            'personale_educatori_effettivi': educatori_effettivi,
            'personale_educatori_previsti': educatori_previsti,
            'personale_polizia_aggiornato_a': personale_polizia_aggiornato_al,
            'personale_amministrativo_aggiornato_al': personale_amministrativo_aggiornato_al,
        }
        
    return prison_data

In [7]:
data = []

for prison_id in prison_ids:
    success = False
    for attempt in range(5):
        try:
            html_content = await get_html(prison_id, current_month)
            # Parse the html with BeautifulSoup
            soup = BeautifulSoup(html_content, 'html.parser')
            prison_data = get_prison_data(soup, current_day)

            # Append prison_data to data list
            data.append(prison_data)

            success = True
            break  # Break the retry loop if successful

        except Exception as e:
            print(f"Attempt {attempt+1} failed for prison_id {prison_id}. Error: {e}")
            if attempt < 4:  # If not the last attempt, sleep for 10 seconds before retrying
                sleep(10)

    if not success:
        print(f"Failed to fetch data for prison_id {prison_id} after 5 attempts.")

    # Sleep for 3 seconds before making the next request
    sleep(3)

# Convert prison_data_list to a Pandas DataFrame
data_df = pd.DataFrame(data)

Fetching MII179988


/tmp/ipykernel_3266/3915262977.py:22: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  target_span = soup.find('h2', text=header_text)
/tmp/ipykernel_3266/3915262977.py:29: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  target_span = soup.find('h2', text=header_text)
/tmp/ipykernel_3266/3915262977.py:56: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  target_span= soup.find('h2', text='personale polizia penitenziaria aggiornato al')
/tmp/ipykernel_3266/3915262977.py:64: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  target_span= soup.find('h2', text='personale amministrativo aggiornato al')


Fetching MII172610


Fetching MII172320


Fetching MII173712


...

Fetching MII179364


Fetching MII178638


Fetching MII181125


Fetching MII159060


Fetching MII179324


In [8]:
data_df

,id,nome,tipo,posti_regolamentari,posti_non_disponibili,posti_occupati,posti_aggiornati_al,asl,nome_responsabile_asl,cognome_responsabile_asl,...,cognome_direttore,ruolo_direttore,personale_polizia_effettivi,personale_polizia_previsti,personale_amministrativi_effettivi,personale_amministrativi_previsti,personale_educatori_effettivi,personale_educatori_previsti,personale_polizia_aggiornato_a,personale_amministrativo_aggiornato_al
0,MII179988,Reggio Calabria Arghillà,Casa circondariale,294,0,353,17/06/2024,5 Reggio Calabria,Luciano,Lucania,...,Stendardo,Titolare,133,160,12,32,5,8,31/05/2024,31/05/2024
1,MII172610,Brescia Verziano,Casa di reclusione,71,0,123,17/06/2024,Spedali Civili di Brescia,Luigi,Leone,...,NA,NA,77,96,2,0,1,0,31/05/2024,31/05/2024
2,MII172320,Busto Arsizio,Casa circondariale,240,11,440,17/06/2024,Busto Arsizio,Ezia,Iorio,...,Pitaniello,Titolare,180,217,14,21,3,5,31/05/2024,31/05/2024
3,MII173712,Como,Casa circondariale,226,0,433,17/06/2024,Como,Giuseppe,Carrano,...,Rinaldi,Titolare,181,236,15,23,3,6,31/05/2024,31/05/2024
4,MII173747,Cremona,Casa circondariale,394,10,577,17/06/2024,CREMONA,Rossano,Botto,...,Padula,Titolare,169,223,16,23,6,6,31/05/2024,31/05/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,MII179364,Parma,Casa di reclusione,655,27,698,17/06/2024,PARMA,Choroma,FAISSAL,...,Pappalardo,Titolare,355,462,2,0,0,0,31/05/2024,31/05/2024
186,MII178638,Modena,Casa circondariale,372,0,537,17/06/2024,MODENA,Stefano,Petrella,...,Sorrentini,Titolare,210,257,22,27,6,6,31/05/2024,31/05/2024
187,MII181125,Siracusa,Casa circondariale,545,0,669,17/06/2024,NA,NA,NA,...,Tiralongo,Titolare,229,250,22,27,6,6,31/05/2024,31/05/2024
188,MII159060,Arienzo,Casa di reclusione - Gennaro De Angelis,58,2,96,17/06/2024,Caserta,Giuseppe,Nese,...,De Fusco,Titolare,47,49,11,13,2,2,31/05/2024,31/05/2024


In [9]:
old_data_path = Path('../outputs/clean/istituti_data.csv')

if old_data_path.exists():
  old_data = pd.read_csv('../outputs/clean/istituti_data.csv')
  combined_data = pd.concat([old_data, data_df], ignore_index=True)
  combined_data.drop_duplicates(inplace=True)
else:
  combined_data = data_df
  combined_data.drop_duplicates

combined_data.to_csv('../outputs/clean/istituti_data.csv', index=False)